# EDA, Feature Engineering and Modelling for Rotor Temperatures


In this project, I will demonstrate various techniques including data visualization, data exploration, machine learning and machine learning optimizations.


## Table of Content

1. [Explorary Data Analysis](#1)
2. [Feature Engineering](#2)
3. [Model Development](#3)
4. [Results and Discussion](#4)

In [7]:
#Load libraries
library(psych)
library(ggplot2)
library(reshape2)
library(EnvStats)
library(tidyverse)
library(GGally)
library(dplyr)
library(MASS)
library(leaps)
library(lattice)
library(glmnet)
library(stats)
library(fastDummies)
library(car)
library(caret)
library(randomForest)

## 1. Exploratory Data Analysis <a class="anchor" id="1"></a>

In this section of the project, I will explore the dataset, to better understand the attributes and the domain knowledge behind these attributes.

In [11]:
df<-read.csv('pmsm_data.csv',header=TRUE)

#Take a look at the first few rows of df
head(df)

ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,profile_id
0.1176819,-1.040829,-1.0267303,0.9250645,1.2124813,0.3586899,-0.8343090,0.3229330,-0.08831267,4
-0.5547717,-1.087189,-1.2258921,0.5125759,2.0241232,0.2841797,-1.3638294,0.2340145,0.71040170,4
-0.7722329,-1.063652,0.2988575,0.9562448,-0.1402458,-0.2591392,1.0291220,-0.2457111,-1.59260400,4
0.2199584,-1.083475,-1.0261079,0.9278145,1.2124836,0.3574240,-0.8381095,0.3228101,-0.14167030,4
0.4015749,-1.078981,0.3155328,-0.7094008,-0.9518901,-0.2562895,1.0291730,-0.2457109,-1.96488810,4
-0.7697667,-1.089577,0.2999456,0.9589649,-0.1402460,-0.2592790,1.0291333,-0.2457186,-1.77564350,4


## 2. Feature Engineering <a class="anchor" id="2"></a>

## 3. Model Development <a class="anchor" id="3"></a>

## 4. Results and Discussion <a class="anchor" id="4"></a>

In [ ]:
_